In [1]:
import pandas as pd
import numpy as np
import rdkit
import duckdb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.metrics import Precision

In [2]:
#Reading only 100,000 rows for the initial training and playground stuff
train_df = pd.read_csv(r"C:\dta_genes\train.csv", nrows=1000000)

In [3]:
#Converting to RDKit molecules
train_df['molecule'] = train_df['molecule_smiles'].apply(Chem.MolFromSmiles)

In [4]:
# Generate ECFPs
def generate_ecfp(molecule, radius=3, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

train_df['ecfp'] = train_df['molecule'].apply(generate_ecfp)

In [5]:
train_df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x0000022995A...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x0000022995A...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x0000022995A...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x0000022995A...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x0000022995A...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
# One hot encoding protein name
one_hot_encoded = pd.get_dummies(train_df['protein_name'], prefix='Protein: ')
one_hot_encoded = one_hot_encoded.astype(int)
ndfn = train_df.drop('protein_name', axis=1)
ndf = pd.concat([ndfn, one_hot_encoded], axis=1)

In [7]:
print(ndf.columns)

Index(['id', 'buildingblock1_smiles', 'buildingblock2_smiles',
       'buildingblock3_smiles', 'molecule_smiles', 'binds', 'molecule', 'ecfp',
       'Protein: _BRD4', 'Protein: _HSA', 'Protein: _sEH'],
      dtype='object')


In [8]:
list_lengths = ndf['ecfp'].apply(len)
print(list_lengths.unique())

[1024]


In [9]:
# Convert the list column into separate columns
expanded_df = pd.DataFrame(ndf['ecfp'].to_list(), columns=[f'ecfp_{i+1}' for i in range(1024)])

# Combine the expanded DataFrame with the original DataFrame
result_df = pd.concat([ndf, expanded_df], axis=1)

# Drop the original 'ecfp' column
result_df.drop(columns=['ecfp'], inplace=True)


In [10]:
# Filter numeric columns
numeric_columns = result_df.select_dtypes(include='number').columns

# Keep only numeric columns
df_numeric = result_df[numeric_columns]

# Display the DataFrame with only numeric columns
print(df_numeric)

            id  binds  Protein: _BRD4  Protein: _HSA  Protein: _sEH  ecfp_1  \
0            0      0               1              0              0       0   
1            1      0               0              1              0       0   
2            2      0               0              0              1       0   
3            3      0               1              0              0       0   
4            4      0               0              1              0       0   
...        ...    ...             ...            ...            ...     ...   
999995  999995      0               0              0              1       0   
999996  999996      0               1              0              0       0   
999997  999997      0               0              1              0       0   
999998  999998      0               0              0              1       0   
999999  999999      0               1              0              0       0   

        ecfp_2  ecfp_3  ecfp_4  ecfp_5  ...  ecfp_1

In [21]:
pc_list = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14']

for i in range(15, 301):
    pc_list.append(f'PC{i}')

In [22]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

dta_for_pca = df_numeric.drop(columns=['id','binds'])

# Perform PCA
pca = PCA(n_components = 300)  # Specify the number of components you want
pca_result = pca.fit_transform(dta_for_pca)

# Create a DataFrame to store the PCA results
pca_df = pd.DataFrame(data=pca_result, columns=pc_list)

# Optionally, you can access the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained variance ratio:", explained_variance_ratio)

# Optionally, you can access the principal components (eigenvectors)
principal_components = pca.components_
print("Principal components (eigenvectors):", principal_components)

Explained variance ratio: [0.04029114 0.02107074 0.01899829 0.01644077 0.01547588 0.01393259
 0.01383947 0.01244007 0.01177031 0.01091226 0.01011457 0.00938718
 0.00914829 0.00908334 0.00865244 0.00829335 0.00827611 0.00795335
 0.00749006 0.00730513 0.00721486 0.00721484 0.00683411 0.0067788
 0.00617406 0.00605365 0.00591683 0.00580766 0.00565354 0.00550892
 0.00543564 0.00537836 0.00518003 0.0050504  0.00490655 0.00487459
 0.00485784 0.00474376 0.00462776 0.00453419 0.00443893 0.00437646
 0.00433537 0.00422909 0.0041377  0.0041034  0.00405716 0.00395791
 0.00395611 0.00386548 0.00385418 0.00377155 0.00368256 0.00365632
 0.00362053 0.00358888 0.00351156 0.00345422 0.00340777 0.00339281
 0.00333387 0.0033242  0.00325198 0.00321625 0.00319498 0.00318578
 0.0031136  0.00308674 0.00304133 0.00300583 0.00298341 0.00297356
 0.0029566  0.00292815 0.00290894 0.00289074 0.00284654 0.00281531
 0.0027797  0.00274475 0.00273396 0.00270865 0.00270398 0.00266639
 0.00264953 0.00263371 0.00261973 0.0

In [23]:
variance_explained = explained_variance_ratio[:300].sum()
print(f"Variance explained by the components: {variance_explained * 100:.2f}%")

Variance explained by the components: 84.33%


In [24]:
X = df_numeric.drop(columns=['id','binds'])
y = df_numeric[['binds']]

In [12]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
print(X_test.columns)

Index(['Protein: _BRD4', 'Protein: _HSA', 'Protein: _sEH', 'ecfp_1', 'ecfp_2',
       'ecfp_3', 'ecfp_4', 'ecfp_5', 'ecfp_6', 'ecfp_7',
       ...
       'ecfp_1015', 'ecfp_1016', 'ecfp_1017', 'ecfp_1018', 'ecfp_1019',
       'ecfp_1020', 'ecfp_1021', 'ecfp_1022', 'ecfp_1023', 'ecfp_1024'],
      dtype='object', length=1027)


In [14]:
print(y.value_counts())

binds
0        997424
1          2576
Name: count, dtype: int64


In [15]:
# 3. Define the Model
model = Sequential()

# Input layer
model.add(Dense(128, input_dim=1027, activation='relu'))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# 4. Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy', Precision()])

# 5. Train the Model
history = model.fit(X_train, y_train, 
                    validation_split=0.2, 
                    epochs=1000, 
                    batch_size=32)

# 6. Evaluate the Model
loss, accuracy, precision = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')
print(f'Test Precision: {precision}')


c:\Users\sageh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.9972 - loss: 0.0249 - precision: 0.0548 - val_accuracy: 0.9975 - val_loss: 0.0291 - val_precision: 0.0000e+00
Epoch 2/1000
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9972 - loss: 0.0241 - precision: 0.0000e+00 - val_accuracy: 0.9975 - val_loss: 0.0215 - val_precision: 0.0000e+00
Epoch 3/1000
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9973 - loss: 0.0254 - precision: 0.0281 - val_accuracy: 0.9975 - val_loss: 0.0172 - val_precision: 0.0000e+00
Epoch 4/1000
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9972 - loss: 0.0257 - precision: 0.0364 - val_accuracy: 0.9975 - val_loss: 0.0163 - val_precision: 0.0000e+00
Epoch 5/1000
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9972 - loss: 0.0225 - precision: 0.0372 - val_accuracy: 0.9975 - val_loss: 0.0158 - val_precision: 0.0000e+00
Epoch 6/1000
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9973 - l

KeyboardInterrupt: 

In [40]:
# Make predictions on the test set
y_pred_proba = model.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step


In [44]:
print(y_pred_proba)
print(y_test)

[[4.7577636e-05]
 [3.1310352e-04]
 [2.1947371e-03]
 ...
 [5.5799028e-05]
 [2.4714967e-04]
 [4.4312347e-02]]
       binds
75721      0
80184      0
19864      0
76699      0
92991      0
...      ...
32595      0
29313      0
37862      0
53421      0
42410      1

[20000 rows x 1 columns]


In [43]:
# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.06
